In [1]:
import nibabel as nib

input_file = r"d:\Kananat\Data\0_Segmentation\47-4881 L 2014_segmented.nii.gz"
img = nib.load(input_file)
data = img.get_fdata()
affine = img.affine

print(data.shape)

(338, 510, 523)


In [2]:
affine = img.affine
header = img.header

In [3]:
from scipy import ndimage

def fill_holes_3d(volume):
            result = volume.copy()
            
            # Process along each axis
            for axis in range(3):
                # For each slice along the current axis
                slices = [result.take(i, axis=axis) for i in range(result.shape[axis])]
                
                # Fill holes in each 2D slice
                filled_slices = [ndimage.binary_fill_holes(slice) for slice in slices]
                
                # Put the filled slices back into the volume
                for i, filled_slice in enumerate(filled_slices):
                    # Create the appropriate slice object for the current axis
                    if axis == 0:
                        result[i, :, :] = filled_slice
                    elif axis == 1:
                        result[:, i, :] = filled_slice
                    else:  # axis == 2
                        result[:, :, i] = filled_slice
                        
            return result

filled_data = fill_holes_3d(data)

In [ ]:
from skimage import morphology
import numpy as np
import networkx as nx

def find_closest_endpoint(skeleton, reference_point):
    """
    Find the endpoint in the skeleton that is closest to the reference point.
    
    Parameters:
    -----------
    skeleton : numpy.ndarray
        Binary 3D array containing the skeletonized structure
    reference_point : tuple
        (x, y, z) coordinate of the reference point
    
    Returns:
    --------
    closest_endpoint : tuple
        (x, y, z) coordinate of the closest endpoint
    """
    # Create a graph from the skeleton
    G = nx.Graph()
    
    # Get coordinates of skeleton voxels
    points = np.transpose(np.where(skeleton))
    
    # Map each point to a unique node ID
    point_to_node = {}
    for i, point in enumerate(points):
        point_tuple = tuple(point)
        point_to_node[point_tuple] = i
        G.add_node(i, pos=point_tuple)
    
    # Add edges between neighboring voxels
    for point_tuple, node_id in point_to_node.items():
        x, y, z = point_tuple
        # Check 26-connected neighbors
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                for dz in [-1, 0, 1]:
                    if dx == 0 and dy == 0 and dz == 0:
                        continue
                        
                    neighbor = (x + dx, y + dy, z + dz)
                    if neighbor in point_to_node:
                        G.add_edge(node_id, point_to_node[neighbor])
    
    # Find endpoints (nodes with only one connection)
    endpoints = [n for n, d in G.degree() if d == 1]
    
    if not endpoints:
        print("Warning: No endpoints found in the skeleton. Using the centroid instead.")
        # If no endpoints, use the centroid of the skeleton
        coords = np.array(np.where(skeleton)).T
        centroid = tuple(np.mean(coords, axis=0).astype(int))
        return centroid
    
    # Get coordinates of endpoints
    endpoint_coords = [G.nodes[n]['pos'] for n in endpoints]
    
    # Find the endpoint closest to the reference point
    closest_endpoint = min(endpoint_coords, 
                          key=lambda p: np.sqrt((p[0]-reference_point[0])**2 + 
                                              (p[1]-reference_point[1])**2 + 
                                              (p[2]-reference_point[2])**2))
    
    return closest_endpoint

def crop_around_point(image, point, crop_size):

    """
    Crop a region of specified size around a point.
    
    Parameters:
    -----------
    image : numpy.ndarray
        3D array to crop from
    point : tuple
        (x, y, z) coordinate of the center point
    crop_size : int
        Size of the crop region (n) for cropping [x-n:x+n, y-n:y+n, z-n:z+n]
    
    Returns:
    --------
    cropped : numpy.ndarray
        Cropped region
    crop_coords : tuple
        ((x_min, x_max), (y_min, y_max), (z_min, z_max)) coordinates of the crop
    """
    x, y, z = point
    
    # Calculate crop boundaries
    x_min = max(0, x - crop_size)
    x_max = min(image.shape[0], x + crop_size + 1)
    y_min = max(0, y - crop_size)
    y_max = min(image.shape[1], y + crop_size + 1)
    z_min = max(0, z - crop_size)
    z_max = min(image.shape[2], z + crop_size + 1)
    
    # Crop the image
    # Create output with zeros
    result = np.zeros_like(image)
    # Copy only the bounding box region from the original image
    result[x_min:x_max, y_min:y_max, z_min:z_max] = image[x_min:x_max, y_min:y_max, z_min:z_max]

    return result

skeleton = morphology.skeletonize(filled_data)

# Find the endpoint closest to the reference point

# print(f"Finding endpoint closest to reference point {reference_point}...")
reference_point = (data.shape[0]//2, 0, 0)
endpoint = find_closest_endpoint(skeleton, reference_point)
# print(f"Found closest endpoint at {endpoint}")

# Crop the region around the endpoint
# print(f"Cropping region of size {crop_size} around endpoint...")
cropped_image = crop_around_point(data, endpoint, 128)
cropped_image = fill_holes_3d(cropped_image)

output_file = f"cropped_fixed_474881.nii.gz"

# print(f"Saving cropped region to {output_file}...")
cropped_img = nib.Nifti1Image(cropped_image.astype(np.int16), affine, header)
nib.save(cropped_img, output_file)

In [10]:
def edited_segmentation(input_file, output_folder, margin_pixels=5):
    
    img = nib.load(input_file)
    data = img.get_fdata()
    affine = img.affine
    header = img.header

    filled_data = fill_holes_3d(data)

    skeleton = morphology.skeletonize(filled_data)

    reference_point = (data.shape[0]//2, 0, 0)
    endpoint = find_closest_endpoint(skeleton, reference_point)
    # print(f"Found closest endpoint at {endpoint}")

    cropped_image = crop_around_point(data, endpoint, 128)
    cropped_image = fill_holes_3d(cropped_image)

    struct_elem = ndimage.generate_binary_structure(3, 1)
    struct_elem = ndimage.iterate_structure(struct_elem, margin_pixels)
    
    # Dilate the binary volume to create the cover with margin
    covered_data = ndimage.binary_dilation(cropped_image, structure=struct_elem).astype(np.int16)

    output_file_name = input_file.name.replace("_segmented.nii.gz", "_editedSegmentation.nii.gz")
    output_file = output_folder / output_file_name

    # print(f"Saving cropped region to {output_file}...")
    cropped_img = nib.Nifti1Image(covered_data.astype(np.int16), affine, header)
    nib.save(cropped_img, output_file)

In [11]:
from pathlib import Path

input_file = Path(r"d:\Kananat\Data\Last0\0_Segmentation\47-4881 L 2014_segmented.nii.gz")
output_folder = Path(r"d:\Kananat\Data\Last0\1_EditedSegmentation")

edited_segmentation(input_file, output_folder)

In [32]:
from pathlib import Path

input_folder = Path(r"D:\Kananat\Data\0_Segmentation")
input_folder2 = Path(r"D:\Kananat\Data\Last0\0_Segmentation")
output_folder = Path(r"D:\Kananat\Data\Last0\1_EditedSegmentation")

glob1 = set(file.name for file in input_folder.glob("*_segmented.nii.gz"))
glob2 = set(file.name for file in input_folder2.glob("*_segmented.nii.gz"))

unprocessed = glob2 - glob1

i = 0
for file in list(unprocessed):
    file_path = input_folder2 / file
    i += 1
    print(f"{i} Processing {file_path}...")
    edited_segmentation(file_path, output_folder)

1 Processing D:\Kananat\Data\Last0\0_Segmentation\65-4618 L 20241218_segmented.nii.gz...
2 Processing D:\Kananat\Data\Last0\0_Segmentation\54-50 R 20160226_segmented.nii.gz...
3 Processing D:\Kananat\Data\Last0\0_Segmentation\66-22465 R 20240527_segmented.nii.gz...
4 Processing D:\Kananat\Data\Last0\0_Segmentation\67-700034 R 20240127_segmented.nii.gz...
5 Processing D:\Kananat\Data\Last0\0_Segmentation\54-25927 L 20160424_segmented.nii.gz...
6 Processing D:\Kananat\Data\Last0\0_Segmentation\54-25927 R 20160424_segmented.nii.gz...
7 Processing D:\Kananat\Data\Last0\0_Segmentation\56-30067 R 20240131_segmented.nii.gz...
8 Processing D:\Kananat\Data\Last0\0_Segmentation\63-9054 R 20240308_segmented.nii.gz...
9 Processing D:\Kananat\Data\Last0\0_Segmentation\67-9653 L 20240701_segmented.nii.gz...
10 Processing D:\Kananat\Data\Last0\0_Segmentation\63-0010001 L 20240527_segmented.nii.gz...
11 Processing D:\Kananat\Data\Last0\0_Segmentation\65-32395 L 20250123_segmented.nii.gz...
12 Processi